In [1]:
%cd ..

/home/jhkim/workspace/lrquant


/home/jhkim/workspace/lrquant/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import gc
import os
import typing

import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.style as mplstyle

mplstyle.use("fast")

from IPython.display import display


if "CUDA_VISIBLE_DEVICES" in os.environ:
    print(os.environ["CUDA_VISIBLE_DEVICES"])

torch.set_printoptions(precision=4, sci_mode=False)

0,1


In [ ]:
from models.LMClass import LMClass

# model_path = "./huggingface/llama-2-7b-hf"
# # model_path = "./output/base1-lr.25"
lm = LMClass("./huggingface/llama-2-7b-hf")


from datautils import get_loaders

dataloader, _ = get_loaders(
    name="wikitext2",
    nsamples=128,
    seed=2,
    model=lm.model_name,
    seqlen=lm.seqlen,
    cache_dir=f"./cache/wikitext2_128_2/dataloader.cache",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

vocab size:  32000
Loading train from ./cache/wikitext2_128_2/dataloader.cache


In [ ]:
layer_inps = torch.zeros((32, 2048, 4096), dtype=torch.float32)

for i, layer in enumerate(lm.model.model.layers):

    def layer_hook(layer_index):
        def _fn(module, input: tuple[torch.Tensor], output):
            layer_inps[layer_index] = input[0][0].detach().clone()

        return _fn

    layer._forward_hooks.clear()
    layer.register_forward_hook(layer_hook(i))

batch_index = 32
with torch.no_grad():
    lm.model.eval()
    lm.model.to(lm.device)
    lm.model(dataloader[batch_index][0].to(lm.device))
    lm.model.cpu()
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
magnitudes = layer_inps.square().mean(dim=2)  # [32, 2048]
print(magnitudes.shape)
topk = magnitudes.topk(5, dim=1)
display(pd.DataFrame([list(zip(topk.values[i], topk.indices[i])) for i in range(32)]))

torch.Size([32, 2048])


,0,1,2,3,4
0,"(tensor(0.0004), tensor(279))","(tensor(0.0004), tensor(133))","(tensor(0.0004), tensor(255))","(tensor(0.0004), tensor(336))","(tensor(0.0004), tensor(1875))"
1,"(tensor(0.0051), tensor(0))","(tensor(0.0037), tensor(30))","(tensor(0.0021), tensor(14))","(tensor(0.0020), tensor(9))","(tensor(0.0019), tensor(25))"
2,"(tensor(2238.6536), tensor(30))","(tensor(197.0514), tensor(0))","(tensor(0.0030), tensor(231))","(tensor(0.0030), tensor(1505))","(tensor(0.0030), tensor(269))"
3,"(tensor(2245.6538), tensor(30))","(tensor(199.3558), tensor(0))","(tensor(0.0058), tensor(1182))","(tensor(0.0057), tensor(2038))","(tensor(0.0057), tensor(436))"
4,"(tensor(2245.4431), tensor(30))","(tensor(199.3570), tensor(0))","(tensor(0.0123), tensor(521))","(tensor(0.0116), tensor(6))","(tensor(0.0113), tensor(41))"
5,"(tensor(2261.6775), tensor(30))","(tensor(203.1048), tensor(0))","(tensor(0.0201), tensor(1476))","(tensor(0.0188), tensor(1720))","(tensor(0.0182), tensor(142))"
6,"(tensor(2261.5203), tensor(30))","(tensor(203.1003), tensor(0))","(tensor(0.0345), tensor(154))","(tensor(0.0294), tensor(1914))","(tensor(0.0292), tensor(1500))"
7,"(tensor(2261.4641), tensor(30))","(tensor(203.0983), tensor(0))","(tensor(0.0466), tensor(673))","(tensor(0.0461), tensor(154))","(tensor(0.0410), tensor(844))"
8,"(tensor(2261.4285), tensor(30))","(tensor(203.0863), tensor(0))","(tensor(0.0721), tensor(673))","(tensor(0.0693), tensor(124))","(tensor(0.0614), tensor(1125))"
9,"(tensor(2261.2019), tensor(30))","(tensor(203.0826), tensor(0))","(tensor(0.1726), tensor(124))","(tensor(0.1448), tensor(1125))","(tensor(0.1365), tensor(1661))"


In [ ]:
display(pd.DataFrame(layer_inps[2, 0].abs()).sort_values(0, ascending=False).T)
display(pd.DataFrame(layer_inps[2, 30].abs()).sort_values(0, ascending=False).T)

,2533,1415,1512,1076,2298,339,3431,2789,2158,3135,...,3678,2993,1123,1673,936,1322,1423,3139,3834,2525
0,769.0,458.5,39.40625,31.265625,18.078125,17.140625,16.078125,15.796875,13.671875,12.640625,...,0.001556,0.001404,0.001236,0.000885,0.000671,0.000595,0.000587,0.000572,0.000458,0.000252


,2533,1415,1512,2298,3431,2168,3135,339,2393,2789,...,287,2803,1002,141,3308,3234,153,612,2000,585
0,2598.0,1532.0,162.625,90.375,59.28125,58.53125,55.15625,53.59375,49.125,39.84375,...,0.003647,0.003258,0.003212,0.003052,0.002472,0.002136,0.000984,0.000671,0.000336,0.000305


In [48]:
display(pd.DataFrame(layer_inps[:, 30, (2533, 1415, 1512)].abs(), columns=["2533", "1415", "1512"]))

,2533,1415,1512
0,0.002792,0.000652,0.004303
1,0.132446,0.265137,1.275391
2,2598.000000,1532.000000,162.625000
3,2602.000000,1535.000000,162.875000
4,2602.000000,1535.000000,161.125000
5,2612.000000,1540.000000,160.875000
6,2612.000000,1540.000000,159.500000
7,2612.000000,1540.000000,159.000000
8,2612.000000,1540.000000,158.750000
9,2612.000000,1540.000000,156.500000


In [8]:
layer_inps_mean = layer_inps.mean(dim=2, keepdim=True)
layer_inps_std = layer_inps.std(dim=2, keepdim=True)
layer_inps_standardized = (layer_inps - layer_inps_mean) / layer_inps_std

In [9]:
df = pd.DataFrame()
df["std_magnitude"] = layer_inps_standardized[2].square().mean(dim=0)
df["magnitude"] = layer_inps[2].square().mean(dim=0)

display(df.sort_values("std_magnitude", ascending=False).T)
display(df.sort_values("magnitude", ascending=False).T)

,1512,2298,1076,2393,3431,3135,257,490,2789,4071,...,473,391,3389,1411,3178,1320,1389,752,3541,1131
std_magnitude,357.096375,236.074036,103.510880,100.425003,86.699539,73.465218,55.564796,47.705299,45.884495,42.762886,...,0.365263,0.364523,0.362759,0.360973,0.356820,0.355335,0.354259,0.342271,0.342075,0.337695
magnitude,14.303576,4.516786,0.975377,1.428895,1.992132,1.686710,0.246256,0.399051,0.975242,0.660897,...,0.000905,0.000767,0.001044,0.001171,0.005337,0.000971,0.001491,0.001473,0.001243,0.000780


,2533,1415,1512,2298,3431,2168,3135,339,2393,1076,...,1131,1133,3050,3520,391,1223,1485,3530,1144,2906
std_magnitude,4.840537,1.639955,357.096375,236.074036,86.699539,2.816064,73.465218,38.540421,100.425003,103.510880,...,0.337695,0.406294,0.399747,0.395986,0.364523,0.396270,0.395221,0.370193,0.390414,0.372287
magnitude,3584.458740,1248.656250,14.303576,4.516786,1.992132,1.707169,1.686710,1.617715,1.428895,0.975377,...,0.000780,0.000778,0.000777,0.000768,0.000767,0.000763,0.000761,0.000752,0.000746,0.000739


In [53]:
display(pd.DataFrame(layer_inps[2, :, 1512].abs()).sort_values(0, ascending=False).T)
display(pd.DataFrame(layer_inps[2, :, 2298].abs()).sort_values(0, ascending=False).T)
display(pd.DataFrame(layer_inps[2, :, 2533].abs()).sort_values(0, ascending=False).T)
display(pd.DataFrame(layer_inps[2, :, 1415].abs()).sort_values(0, ascending=False).T)

,30,0,664,1645,172,2044,1291,519,1447,1999,...,658,1487,1792,199,1915,1522,155,1720,1572,1011
0,162.625,39.40625,1.852539,1.806641,1.804688,1.796875,1.78125,1.75,1.743164,1.740234,...,0.110107,0.09375,0.085693,0.084106,0.054688,0.04834,0.047729,0.04126,0.009766,0.009521


,30,0,1306,926,507,122,682,1667,454,1097,...,1937,264,1353,155,49,1189,1298,1486,1769,1412
0,90.375,18.078125,1.49707,1.496094,1.486328,1.485352,1.478516,1.46875,1.46582,1.463867,...,0.011597,0.011475,0.009644,0.009277,0.008301,0.007935,0.007568,0.002197,0.001465,0.000427


,30,0,1965,1426,391,843,1438,1964,390,32,...,2041,1121,131,1850,72,2046,177,1474,1451,56
0,2598.0,769.0,0.177734,0.168701,0.166138,0.164185,0.153564,0.148438,0.138916,0.137939,...,0.000763,0.000717,0.000519,0.000401,0.00032,0.000275,0.000222,0.000153,0.000092,0.0


,30,0,383,382,384,1418,385,1419,1420,1958,...,1194,440,1218,780,1461,1642,642,1443,1396,689
0,1532.0,458.5,0.117126,0.116272,0.112427,0.105835,0.105713,0.104858,0.101562,0.101562,...,0.000153,0.000122,0.000122,0.000114,0.000092,0.000092,0.000092,0.000061,0.000061,0.000031


In [11]:
def get_max_df(single_layer_inps: torch.Tensor):
    max_df = pd.DataFrame(single_layer_inps.abs().max(dim=0)).transpose()
    max_df.columns = ["max", "channel_idx"]
    max_df["max"] = max_df["max"].apply(lambda x: x.item())
    max_df["channel_idx"] = max_df["channel_idx"].apply(lambda x: x.item())
    max_df["std"] = single_layer_inps.std(dim=0)
    # for i, row in enumerate(max_df.iterrows()):
    #     channel_idx = row[1]["channel_idx"]
    #     max_df.at[row[0], "value"] = single_layer_inps[i][channel_idx].item()
    #     max_df.at[row[0], "std"] = single_layer_inps[i].std().item()

    max_df = max_df.sort_values("max", ascending=False)
    # max_df = max_df.drop_duplicates(subset=["channel_idx"])
    return max_df

def get_mean_df(single_layer_inps: torch.Tensor):
    mag_df = pd.DataFrame(single_layer_inps.abs().mean(dim=0))
    mag_df.columns = ["mean_abs"]
    mag_df["mean_abs"] = mag_df["mean_abs"].apply(lambda x: x.item())
    mag_df["std"] = single_layer_inps.std(dim=0)
    # for i, row in enumerate(mag_df.iterrows()):
    #     mag_df.at[row[0], "std"] = single_layer_inps[i].std().item()
    mag_df = mag_df.sort_values("mean_abs", ascending=False)
    return mag_df

for i in range(33):
    max_df = get_max_df(layer_inps[i])
    print(f"Layer {i} max df", max_df, sep='\n')
    
    mag_df = get_mean_df(layer_inps[i])
    print(f"Layer {i} magnitude df", mag_df, sep='\n')

Layer 0 max df
           max  channel_idx       std
2459  0.128906            0  0.038803
1571  0.125000            9  0.039774
2403  0.121582           27  0.035343
1307  0.118652          108  0.035887
2622  0.112793           27  0.030817
...        ...          ...       ...
2393  0.037109           98  0.013993
2316  0.036865          797  0.012872
2235  0.035889          874  0.008518
1218  0.034668          322  0.011758
1415  0.023682          182  0.007440

[4096 rows x 3 columns]


AttributeError: 'float' object has no attribute 'item'

In [ ]:
def find_outlier_channels(activation, threshold=20):
    mean = activation.mean(dim=(0, 1))
    std = activation.std(dim=(0, 1))
    z_scores = (activation - mean) / std
    outliers = torch.where(z_scores > threshold)
    unique_channels = torch.unique(outliers[2])
    return unique_channels.tolist()
print(find_outlier_channels(layer_inps))

[575, 580, 1076, 2350, 2533, 2944, 2958, 3135, 3431, 3994]


In [ ]:
# standardize the activations
layer_inps_std = (layer_inps - layer_inps.mean(dim=(0, 1), keepdim=True)) / layer_inps.std(dim=(0, 1), keepdim=True)

for i in range(33):
    max_df = get_max_df(layer_inps_std[i])
    print(f"Layer {i} max df", max_df, sep='\n')
    
    # mag_df = get_magnitude_df(layer_inps_std[i])
    # print(f"Layer {i} magnitude df", mag_df, sep='\n')

Layer 0 max df
           max  channel_idx       std
623   1.016602         1076  0.166992
978   1.015625         1076  0.166626
225   1.015625         1076  0.166626
679   1.015625         1076  0.167358
1525  1.014648         1076  0.166992
...        ...          ...       ...
340   1.000977         1076  0.166626
915   1.000977         1076  0.166382
886   1.000977         1076  0.166504
1422  1.000977         1076  0.166992
422   1.000000         1076  0.167358

[2048 rows x 3 columns]
Layer 1 max df
           max  channel_idx       std
1211  1.061523         1076  0.168823
1930  1.053711         1076  0.168457
1932  1.050781         1076  0.168213
1921  1.048828         1076  0.168213
1726  1.048828         1076  0.167603
...        ...          ...       ...
125   0.959473         1076  0.166870
126   0.958496         1076  0.166748
1172  0.958008         1076  0.166382
1167  0.957031         1076  0.166016
127   0.956543         1076  0.166748

[2048 rows x 3 columns]
Layer 2 

In [12]:
print(layer_inps_std[3][:,2533].abs().sort(descending=True))
print(layer_inps_std[3][:,1415].abs().sort(descending=True))

IndexError: index 2533 is out of bounds for dimension 1 with size 1

In [13]:
for i in range(33):
    print(layer_inps[i].mean(dim=0).abs().sort(descending=True))


torch.return_types.sort(
values=tensor([    0.0543,     0.0524,     0.0477,  ...,     0.0000,     0.0000,
            0.0000]),
indices=tensor([3964, 1411, 4030,  ...,  177, 1447,  549]))
torch.return_types.sort(
values=tensor([    0.8999,     0.6569,     0.1518,  ...,     0.0000,     0.0000,
            0.0000]),
indices=tensor([1512, 2298, 2944,  ..., 3084, 1455, 3675]))
torch.return_types.sort(
values=tensor([    1.6148,     0.9605,     0.6345,  ...,     0.0000,     0.0000,
            0.0000]),
indices=tensor([2533, 1415, 1512,  ..., 1411, 2467,  441]))
torch.return_types.sort(
values=tensor([    1.6658,     0.9670,     0.8359,  ...,     0.0000,     0.0000,
            0.0000]),
indices=tensor([2533, 1415, 1512,  ...,  216, 2288,  983]))
torch.return_types.sort(
values=tensor([    1.6293,     1.3685,     0.9450,  ...,     0.0000,     0.0000,
            0.0000]),
indices=tensor([2533, 1512, 1415,  ..., 1171,  453, 3779]))
torch.return_types.sort(
values=tensor([    1.7276,     1.37

IndexError: index 32 is out of bounds for dimension 0 with size 32